In [1]:
import numpy as np 
import pandas as pd 

from subprocess import check_output

diff=pd.read_csv("D:\Symptom checker\diffsydiw.csv")
sym=pd.read_csv("D:\Symptom checker\sym_t.csv")
dia=pd.read_csv("D:\Symptom checker\dia_t.csv")

# Preprossing the data

dia['diagnose'] = dia['diagnose'].str.replace('\x0b', ': ')
# Merging 
sd_diff=diff.merge(sym, left_on='syd', right_on='syd')
sd_diff=sd_diff.merge(dia, left_on='did', right_on='did')
dia.head()
diff.head()

,syd,did,wei
0,1.0,163.0,2.0
1,1.0,164.0,2.0
2,1.0,165.0,1.0
3,1.0,187.0,2.0
4,1.0,306.0,2.0


In [2]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from scipy.sparse import coo_matrix, csr_matrix

def read_data(filename):
    # read in triples of syd/did/wei from the input dataset
    data = pd.read_csv(filename) 

    data=data.dropna(axis=0, how='any')  #drop nan
    data['wei']=data['wei']+1 # to avoid zero values 
    data.head()

    # map each did and sys to a unique numeric value
    data['syd'] = data['syd'].astype("category")
    data['did'] = data['did'].astype("category")

    # create a sparse matrix of all the syd/wei
    wei = coo_matrix((data['wei'].astype(float),
                       (data['did'].cat.codes.copy(),
                        data['syd'].cat.codes.copy())))

    return data, wei,data.groupby(['did']).wei.sum(),data['syd'].cat.codes.copy()

data,matrix,songsd,user=read_data("D:\Symptom checker\diffsydiw.csv")
data.head(10)


C:\Users\Tappa\AppData\Local\Temp\ipykernel_1416\540058217.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return data, wei,data.groupby(['did']).wei.sum(),data['syd'].cat.codes.copy()


,syd,did,wei
0,1.0,163.0,3.0
1,1.0,164.0,3.0
2,1.0,165.0,2.0
3,1.0,187.0,3.0
4,1.0,306.0,3.0
5,1.0,307.0,2.0
6,1.0,308.0,3.0
7,1.0,309.0,3.0
8,1.0,354.0,2.0
9,1.0,401.0,2.0


In [3]:
from sklearn.preprocessing import normalize


def cosine(wei):
    normalized = normalize(wei)
    return normalized.dot(normalized.T)


def bhattacharya(wei):
    plays.data = np.sqrt(wei.data)
    return cosine(wei)


def ochiai(wei):
    plays = csr_matrix(wei)
    plays.data = np.ones(len(wei.data))
    return cosine(wei)


def bm25_weight(data, K1=1.2, B=0.8):
    
    # Weighs each row of the matrix data by BM25 weighting 
 
    N = float(data.shape[0])
    idf = np.log(N / (1 + np.bincount(data.col)))

    # calculate length_norm per document
    row_sums = np.squeeze(np.asarray(data.sum(1)))
    average_length = row_sums.sum() / N
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    ret = coo_matrix(data)
    ret.data = ret.data * (K1 + 1.0) / (K1 * length_norm[ret.row] + ret.data) * idf[ret.col]
    return ret


def bm25(wei):
    wri = bm25_weight(wei)
    return wei.dot(wei.T)

def get_largest(row, N=10):
    if N >= row.nnz:
        best = zip(row.data, row.indices)
    else:
        ind = np.argpartition(row.data, -N)[-N:]
        best = zip(row.data[ind], row.indices[ind])
    return sorted(best, reverse=True)


def calculate_similar_diseases(similarity, diseases, diseaseid):
    neighbours = similarity[disease]
    top = get_largest(neighbours)
    return [(diseases[other], score, i) for i, (score, other) in enumerate(top)]


#songsd = dict(enumerate(data['song'].cat.categories))
user_count = data.groupby('syd').size()
#to_generate = sorted(list(songsd), key=lambda x: -user_count[x])

similarity = bm25_weight(matrix)

C:\Users\Tappa\AppData\Local\Temp\ipykernel_1416\3189163377.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_count = data.groupby('syd').size()


In [47]:
print(bm25_weight(coo_matrix(matrix)))

  (151, 0)	3.724880552518494
  (152, 0)	5.201207185057583
  (153, 0)	4.0480770951538965
  (175, 0)	4.139583664150382
  (286, 0)	4.299129020200215
  (287, 0)	3.25156553443722
  (288, 0)	3.991456374767475
  (289, 0)	3.3857013063189174
  (331, 0)	4.2811248660446575
  (369, 0)	3.479797249832829
  (377, 0)	5.561924745405236
  (468, 0)	2.990082253331463
  (498, 0)	4.3836041438668225
  (654, 0)	5.561924745405236
  (50, 1)	4.246561580895969
  (167, 1)	3.7339423779864385
  (224, 1)	3.6889530606849386
  (359, 1)	4.157513973642382
  (491, 1)	4.219885567344989
  (492, 1)	5.284732646377084
  (507, 1)	3.4300205322960475
  (548, 1)	5.48236426037233
  (721, 1)	2.5903456058886856
  (723, 1)	4.929340868145222
  (39, 2)	7.7104883250391545
  :	:
  (1, 260)	7.910168604990296
  (118, 258)	6.3771365188506035
  (391, 258)	3.8289317454265017
  (141, 262)	4.484882253426113
  (264, 250)	5.949900595253816
  (547, 259)	2.5014629841842773
  (187, 259)	3.9861119774493745
  (659, 259)	6.7790111524811145
  (658, 259)	

In [24]:
print(sym)
sym[sym['syd'].isin(list(songsd.index))]

     syd                          symptom
0      1             Upper abdominal pain
1      2             Lower abdominal pain
2      3      Abscess (Collection of pus)
3      4                    Alcohol abuse
4      5            Anxiety (Nervousness)
..   ...                              ...
267  295                 Nipple discharge
268  301  Shoulder stiffness or tightness
269  303       Arm stiffness or tightness
270  304              High blood pressure
271  306                 High blood sugar

[272 rows x 2 columns]


,syd,symptom
0,1,Upper abdominal pain
1,2,Lower abdominal pain
2,3,Abscess (Collection of pus)
3,4,Alcohol abuse
4,5,Anxiety (Nervousness)
...,...,...
266,294,Confusion and headache
267,295,Nipple discharge
269,303,Arm stiffness or tightness
270,304,High blood pressure


In [37]:
from scipy.sparse.linalg import svds

Ur, Si, VTr = svds(bm25_weight(coo_matrix(matrix)), k=100)
print(Ur.shape, Si.shape, VTr.shape,user.shape,matrix.shape,
      data.shape,songsd.shape,user_count.shape)
VTr=pd.DataFrame(VTr)

(1145, 100) (100,) (100, 272) (5371,) (1145, 272) (5371, 3) (1145,) (272,)


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

Sddf=pd.DataFrame(cosine_similarity(Ur,VTr.T),
                  columns=user_count.index,index=list(songsd.index))
Sddf.to_csv('D:\Symptom checker\Sddf.csv')

Sydi=pd.DataFrame(cosine_similarity(Ur,VTr.T))

In [48]:
Sydi.head()

,0,1,2,3,4,5,6,7,8,9,...,262,263,264,265,266,267,268,269,270,271
0,-0.038147,0.042361,0.015335,-0.005820,-0.012366,-0.045712,0.478176,-0.016260,-0.055674,0.037448,...,-0.107003,0.021731,-0.028037,-0.001658,-0.005130,0.010329,0.000496,-0.025334,0.000486,-0.022492
1,-0.048048,-0.047444,-0.024307,-0.005287,0.000176,0.015011,-0.018259,0.009472,0.013929,0.000432,...,-0.009741,-0.011940,0.008559,-0.008553,0.000129,0.002911,-0.009497,-0.023477,-0.010001,-0.015174
2,0.154661,0.425991,-0.027339,0.060376,-0.015116,-0.045309,-0.049484,0.035197,-0.029371,-0.038369,...,0.083395,0.015455,0.044322,-0.022521,0.016632,0.071560,-0.072679,-0.047323,-0.025664,-0.000058
3,0.033917,-0.098229,-0.051431,-0.014986,-0.000067,0.449473,0.007464,0.010383,-0.041795,0.010354,...,0.116953,-0.084778,-0.006112,-0.025841,0.013883,0.049498,-0.095373,-0.095418,-0.025763,-0.043704
4,0.018518,0.037765,-0.057741,-0.065873,0.030747,-0.001378,-0.012344,0.014485,-0.003346,-0.030964,...,0.015449,0.033380,0.009893,-0.022633,0.043962,0.006446,0.019427,0.026308,0.020922,0.039194


In [ ]:
# 'dia' DataFrame is assumed to contain 'did' and 'diagnose' columns as in the previous code
treatments= pd.read_csv("D:\Symptom checker\prec_t.csv")
# 1. Take symptoms as input from the user
user_input = input("Enter symptoms separated by commas: ")
user_symptoms = [symptom.strip() for symptom in user_input.split(',')]

# 2. Recognize the indices of the input symptoms
recognized_indices = []
for user_symptom in user_symptoms:
    # Assuming 'syd' is the column containing symptom indices in the 'sym' DataFrame
    recognized_index = sym[sym['symptom'] == user_symptom]['syd'].values
    if len(recognized_index) > 0:
        recognized_indices.append(recognized_index[0])

# 3. Recommend diseases based on recognized indices
if len(recognized_indices) > 0:
    print("\nRecognized Symptoms:")
    for index in recognized_indices:
        print(f"Symptom: {sym[sym['syd'] == index]['symptom'].values[0]}")

    print('\nTop 5 related diseases probability:')

    # Combine similarity scores for recognized indices
    combined_similarity = Sddf[recognized_indices].mean(axis=1)

    # Recommend diseases based on combined similarity
    top_diseases = combined_similarity.sort_values(ascending=False)

    for disease_id, similarity_score in top_diseases.head(5).items():
    # Find the corresponding diagnose for the disease_id
      diagnose = dia[dia['did'] == disease_id]['diagnose'].values[0]
      print(f"\nPossible Disease: {diagnose}")

      # Include the similarity score in the output
      print(f"Similarity Score: {similarity_score}")

     # Search for treatment in 'prec_t.csv'
      treatment = treatments[treatments['diagnose'] == diagnose]['pid'].values
      if len(treatment) > 0:
        print(f"Recommended Treatment: {treatment[0]}")
      else:
        print("No specific treatment recommendation found.")

else:
    print("No recognized symptoms.")